In [ ]:
from itertools import product
import numpy as np

In [ ]:
def update_inner_volumes(inner_volumes, slices, sub):
    if np.all(sub):
        inner_vol = np.prod(np.array(sub.shape) - 1)
        for pt_ in product(*slices):
            inner_volumes[pt_].append(inner_vol)


def max_inner_volumes(mat):
    dims = mat.shape
    pts = product(*[range(dim) for dim in dims])
    inner_volumes = {pt: [] for pt in pts}
#     zeros = np.where(~mat)

    lst_bounds = [product(range(dim + 1), repeat=2) for dim in dims]
    for bounds in product(*lst_bounds):
        if all(bound[0] < bound[1] for bound in bounds):
            slices = [np.arange(*bound) for bound in bounds]
#             if not any(
#                 all(coor in slic for coor, slic in zip(zero, slices))
#                 for zero in zip(*zeros)):
            sub = mat[np.ix_(*slices)]
            update_inner_volumes(inner_volumes, slices, sub)

    return np.array([
        (max(vols) if vols else 0)
        for vols in inner_volumes.values()]).reshape(dims)

In [ ]:
def max_inner_volumes_simplified(mat):
    dims = mat.shape
    inner_volumes = np.zeros(dims, dtype=int)

    for left in np.ndindex(dims):
        for right in product(*[range(l + 1, dim + 1) for l, dim in zip(left, dims)]):
            slices = tuple(slice(l, r) for l, r in zip(left, right))
            # shape = tuple(np.array(right) - np.array(left))
            sub = mat[slices]
            # print(slices, sub.shape, shape)
            if np.all(sub):
                inner_volume = np.prod(np.array(sub.shape) - 1)
                inner_volumes[slices] = np.maximum(inner_volumes[slices], inner_volume)

    return inner_volumes

In [ ]:
def max_inner_volumes_stacked(mat):
    # loop at constant shape; memory is more used vs np.all is more multiprocessed
    dims = mat.shape
    inner_volumes = np.zeros(dims, dtype=int)

    for shape_ in np.ndindex(dims):
        shape = np.array(shape_) + 1
        slices = [
            tuple(slice(r - s, r) for s, r in zip(shape, right))
            for right in product(*[range(s, dim + 1) for s, dim in zip(shape, dims)])
        ]
        subs = np.array([mat[slic].reshape(-1) for slic in slices])
        for ind in np.where(np.all(subs, axis=1))[0]:
            inner_volume = np.prod(shape - 1)
            inner_volumes[slices[ind]] = np.maximum(inner_volumes[slices[ind]], inner_volume)

    return inner_volumes

In [ ]:
funcs = [max_inner_volumes, max_inner_volumes_simplified, max_inner_volumes_stacked]

In [ ]:
expl = np.array([False, True, True, False, False])
res = [func(expl) for func in funcs]
assert all(np.array_equal(a, b) for a, b in zip(res, res[1:]))

In [ ]:
expl = np.array([
    [False, True, True,  False, False],
    [True,  True, True,   True, False],
    [False, True, False, False, False],
])

res = [func(expl) for func in funcs]
assert all(np.array_equal(a, b) for a, b in zip(res, res[1:]))

In [ ]:
expl = np.random.rand(15, 15, 15) < 0.5

In [ ]:
%%time
res_0 = funcs[0](expl)

In [ ]:
%%time
res_1 = funcs[1](expl)

In [ ]:
%%time
res_2 = funcs[2](expl)

In [ ]:
res = [res_0, res_1, res_2]
assert all(np.array_equal(a, b) for a, b in zip(res, res[1:]))

In [ ]:
def convolve_nd_no_padding(kernel, mat):
    kernel_shape = kernel.shape
    mat_shape = mat.shape
    assert all(dim % 2 == 1 for dim in kernel_shape)
    half_kernel_shape = tuple((dim - 1) // 2 for dim in kernel_shape)

    convs = np.zeros(mat_shape, dtype=float)
    for index in np.ndindex(mat_shape):
        sub_mat = mat[tuple(
            slice(
                max(index[dim] - 1, 0),
                index[dim] + half_kernel_shape[dim] + 1
            )
            for dim in range(mat.ndim)
        )]
        sub_kernel = kernel[tuple(
            slice(
                max(half_kernel_shape[dim] - index[dim], 0),
                mat_shape[dim] + half_kernel_shape[dim] - index[dim]
            )
            for dim in range(kernel.ndim)
        )]
        convs[index] = np.multiply(sub_mat, sub_kernel / sub_kernel.sum()).sum()
    return convs